In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
from scipy.stats import norm
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import datetime, timedelta
import folium
import plotly.express as px

In [2]:
token = 'd-bqq4Fx4pjuztAPQkYS'

In [7]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'https://remobsapi.herokuapp.com/api/v1/data_stations/last?start_date=2021-10-26T00:00:00&end_date=2021-10-26T06:00:00&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)

In [15]:
df.lat[df.lat.notna()]

659   -22.929699
642   -22.944264
643   -22.947623
644   -22.935115
645   -22.928593
         ...    
235   -10.723100
236   -14.181900
237   -13.009500
229   -14.208100
700   -20.258100
Name: lat, Length: 695, dtype: float64

In [45]:
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)

In [46]:
df['coordinates'] = coordinates

In [47]:
def haversine(row, lon1, lat1):
    lon2 = row['lon']
    lat2 = row['lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    mn = 3437.9 * c
    return mn

In [48]:
spotter = df

In [49]:
spotter['distance'] = spotter.apply(lambda row: haversine(row, spotter['lon'].iloc[0], spotter['lat'].iloc[0]), axis=1)

In [50]:
spotter['veloc'] = spotter['distance'].diff()/(spotter['date_time'].diff().dt.total_seconds()/3600)

In [51]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'https://remobsapi.herokuapp.com/api/v1/data_buoys?buoy=2&start_date=2021-10-18&end_date={time_now}&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)

In [52]:
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)
df['coordinates'] = coordinates
df['distance'] = df.apply(lambda row: haversine(row, df['lon'].iloc[0], df['lat'].iloc[0]), axis=1)

In [53]:
df['veloc'] = df['distance'].diff()/(df['date_time'].diff().dt.total_seconds()/3600)
df1 = df.copy()

In [54]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'https://remobsapi.herokuapp.com/api/v1/data_buoys?buoy=2&start_date=2021-10-14&end_date=2021-10-18&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)
df['coordinates'] = coordinates
df['distance'] = df.apply(lambda row: haversine(row, df['lon'].iloc[0], df['lat'].iloc[0]), axis=1)
df['veloc'] = df['distance'].diff()/(df['date_time'].diff().dt.total_seconds()/3600)

In [55]:
df1 = pd.concat([df, df1])

In [56]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'https://remobsapi.herokuapp.com/api/v1/data_buoys?buoy=2&start_date=2021-10-10&end_date=2021-10-14&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)
df['coordinates'] = coordinates
df['distance'] = df.apply(lambda row: haversine(row, df['lon'].iloc[0], df['lat'].iloc[0]), axis=1)
df['veloc'] = df['distance'].diff()/(df['date_time'].diff().dt.total_seconds()/3600)

In [57]:
pd.concat([df, df1]).to_csv('dados.csv')

In [59]:
m = folium.Map(location=spotter['coordinates'].iloc[0], zoom_start=11)

for index, row in df.iterrows():
    popup = str(row['date_time']) + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
    folium.Marker(row['coordinates'], tooltip=popup).add_to(m)


for index, row in spotter.iterrows():
    popup = str(row['date_time']) + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
    folium.Marker(row['coordinates'], tooltip=popup, icon=folium.Icon(color='red')).add_to(m)
folium.Marker(
    spotter['coordinates'].iloc[-1],
    tooltip=popup,
    icon=folium.Icon(color='red')).add_to(m)    

deployment_loc = [-25.508, -42.736]
folium.Circle(deployment_loc, radius=1300).add_to(m)

m.save(outfile= "bmo_deriva.html")


m

In [36]:
df1.iloc[0:-1].tail()

,id,buoy_id,lat,lon,date_time,battery,compass,flag_compass,flood,rh,...,flag_wvspread1,pk_dir,flag_pk_dir,pk_wvspread,flag_pk_wvspread,mean_tp,flag_mean_tp,coordinates,distance,veloc
5,7929,2,-25.5056,-42.7245,2021-10-22 16:00:00,13.1,317.0,0,NaN,60.6,...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[-25.5056, -42.7245]",0.299272,-0.028778
4,7930,2,-25.5054,-42.7247,2021-10-22 17:00:00,13.2,291.0,0,NaN,63.1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[-25.5054, -42.7247]",0.283457,-0.015815
3,7931,2,-25.5055,-42.7247,2021-10-22 18:00:00,13.0,354.0,0,NaN,59.2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[-25.5055, -42.7247]",0.288680,0.005223
2,7932,2,-25.5057,-42.7245,2021-10-22 19:00:00,13.0,319.0,0,NaN,59.5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[-25.5057, -42.7245]",0.304460,0.015780
1,7933,2,-25.5060,-42.7244,2021-10-22 20:00:00,12.9,38.0,0,NaN,67.4,...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[-25.506, -42.7244]",0.322795,0.018335


In [40]:
m = folium.Map(location=spotter['coordinates'].iloc[0], zoom_start=11)

for index, row in df1.iterrows():
    popup = str(row['date_time']) + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
    folium.Marker(row['coordinates'], tooltip=popup).add_to(m)



# for index, row in spotter.iterrows():
#     popup = str(row['date_time']) + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
#     folium.Marker(row['coordinates'], tooltip=popup, icon=folium.Icon(color='red')).add_to(m)
# folium.Marker(
#     spotter['coordinates'].iloc[-1],
#     tooltip=popup,
#     icon=folium.Icon(color='red')).add_to(m)    

deployment_loc = [-25.508, -42.736]
folium.Circle(deployment_loc, radius=1300).add_to(m)

m.save(outfile= "bmo_posição.html")


m

In [54]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'https://remobsapi.herokuapp.com/api/v1/data_buoys?buoy=2&start_date=2021-01-01&end_date={time_now}&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)

In [9]:
spotter = df[['date_time', 'lat', 'lon','swvht1', 'flag_swvht1', 'swvht2', 'flag_swvht2',
       'mxwvht1', 'flag_mxwvht1', 'tp1', 'flag_tp1', 'tp2', 'flag_tp2',
       'wvdir1', 'flag_wvdir1', 'wvdir2', 'flag_wvdir2', 'wvspread1',
       'flag_wvspread1', 'pk_dir', 'flag_pk_dir', 'pk_wvspread',
       'flag_pk_wvspread', 'mean_tp', 'flag_mean_tp', 'sst',
       'flag_sst']]

In [63]:
df.loc[df['flag_tp2'] > 0, 'tp2'] = np.nan


In [11]:
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)

In [12]:
spotter['coordinates'] = coordinates

In [13]:
def haversine(row, lon1, lat1):
    lon2 = row['lon']
    lat2 = row['lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    mn = 3437.9 * c
    return mn

In [14]:
spotter['distance'] = spotter.apply(lambda row: haversine(row, spotter['lon'].iloc[0], spotter['lat'].iloc[0]), axis=1)

In [15]:
spotter['veloc'] = spotter['distance'].diff()/(spotter['date_time'].diff().dt.total_seconds()/3600)

In [53]:
m = folium.Map(location=spotter['coordinates'].iloc[0], zoom_start=8)

icon = folium.features.CustomIcon(folium.Icon(color="red"),
                                      icon_size=(14, 14))

for index, row in spotter.iterrows():
    popup = str(row['date_time']) + ' - onda ' + \
     str(row['swvht1']) + 'm' + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
    folium.Marker(row['coordinates'], tooltip=popup).add_to(m)
folium.Marker(
    spotter['coordinates'].iloc[-1],
    tooltip=popup,
    icon=icon,
).add_to(m)    

folium.Circle(spotter['coordinates'].iloc[0], radius=100).add_to(m)
m

TypeError: Object of type Icon is not JSON serializable

In [17]:
spotter['distance'].tail()

578    0.649660
579    0.633098
580    0.615757
581    0.598596
582    0.581629
Name: distance, dtype: float64